In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [2]:
train_data = pd.read_csv('../data/train_data_modified.csv')
train_data.drop(['CategoryCode', 'Week'], axis = 1, inplace = True)
train_data.head()

,ItemCode,id,WeeklySales
0,174436,1,35
1,118033,1,7
2,20824,1,53
3,1061341,1,18
4,76399,1,21


In [3]:
valid_data = pd.read_csv('../data/validation_data.csv')
valid_data.head()

,CategoryCode,ItemCode,Week,WeeklySales
0,category_2,1044502,w1,11
1,category_2,1105009,w1,11
2,category_2,913561,w4,5
3,category_1,1048975,w4,30
4,category_1,17287,w2,60


In [4]:
def week_id_valid(week):
    if week=="w1":
        return 1
    elif week=="w2":
        return 2
    elif week=="w3":
        return 3
    else: return 4

valid_data['id'] = [week_id_valid(week)+20 for week in valid_data['Week']]
valid_data

,CategoryCode,ItemCode,Week,WeeklySales,id
0,category_2,1044502,w1,11,21
1,category_2,1105009,w1,11,21
2,category_2,913561,w4,5,24
3,category_1,1048975,w4,30,24
4,category_1,17287,w2,60,22
...,...,...,...,...,...
365,category_2,124954,w2,43,22
366,category_2,40759,w1,48,21
367,category_1,1090303,w1,19,21
368,category_2,1090276,w3,6,23


In [5]:
valid_data.sort_values(by = 'id', inplace = True)
valid_data.head()

,CategoryCode,ItemCode,Week,WeeklySales,id
0,category_2,1044502,w1,11,21
142,category_1,48940,w1,22,21
269,category_1,35584,w1,15,21
138,category_2,59749,w1,33,21
137,category_2,132028,w1,112,21


In [6]:
valid_data.drop(['CategoryCode', 'Week'], axis = 1, inplace = True)

In [23]:
train_val_data = pd.concat([train_data, valid_data])
train_val_data.tail()

,ItemCode,id,WeeklySales
139,118033,24,12
140,1098502,24,4
268,1010068,24,19
146,211309,24,23
369,3418,24,69


In [24]:
melt2 = train_val_data.copy()

In [9]:
melt2['Last_Week_Sales'] = melt2.groupby(['ItemCode'])['WeeklySales'].shift()

In [11]:
#melt2['Last_Week_Diff'] = melt2.groupby(['ItemCode'])['Last_Week_Sales'].diff()

In [25]:
melt2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4142 entries, 0 to 369
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   ItemCode     4142 non-null   int64
 1   id           4142 non-null   int64
 2   WeeklySales  4142 non-null   int64
dtypes: int64(3)
memory usage: 129.4 KB


In [26]:
melt2 = melt2.dropna()
melt2.tail()

,ItemCode,id,WeeklySales
139,118033,24,12
140,1098502,24,4
268,1010068,24,19
146,211309,24,23
369,3418,24,69


In [27]:
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_log_error(ytrue, ypred))

In [16]:
mean_error = []
for week in range(21,25):
    train = melt2[melt2['id'] < week]
    val = melt2[melt2['id'] == week]
    p = val['Last_Week_Sales'].values

    error = rmsle(val['WeeklySales'].values, p)
    print('Week %d - Error %.5f' % (week, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

Week 21 - Error 0.63727
Week 22 - Error 0.62979
Week 23 - Error 0.69828
Week 24 - Error 0.67142
Mean Error = 0.65919


In [31]:
mean_error = []
for week in range(21,25):
    train = melt2[melt2['id'] < week]
    val = melt2[melt2['id'] == week]

    xtr, xts = train.drop(['WeeklySales'], axis=1), val.drop(['WeeklySales'], axis=1)
    ytr, yts = train['WeeklySales'].values, val['WeeklySales'].values

    mdl = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
    mdl.fit(xtr, ytr)

    p = mdl.predict(xts)

    error = rmsle(yts, p)
    print('Week %d - Error %.5f' % (week, error))
    mean_error.append(error)
print('Mean Error = %.5f' % np.mean(mean_error))

Week 21 - Error 0.59528
Week 22 - Error 0.57015
Week 23 - Error 0.63792
Week 24 - Error 0.59466
Mean Error = 0.59950


### Submission dataset

In [32]:
test_data = pd.read_csv('../data/test_data.csv')
test_data.head()

,CategoryCode,ItemCode,Week,PredictedSales
0,category_1,43738,w4,NaN
1,category_2,1006090,w1,NaN
2,category_2,1076929,w4,NaN
3,category_1,1081321,w3,NaN
4,category_2,216151,w4,NaN


In [33]:
def week_id_test(week):
    if week=="w1":
        return 1
    elif week=="w2":
        return 2
    elif week=="w3":
        return 3
    else: return 4

test_data['id'] = [week_id_valid(week) for week in test_data['Week']]
test_data

,CategoryCode,ItemCode,Week,PredictedSales,id
0,category_1,43738,w4,NaN,4
1,category_2,1006090,w1,NaN,1
2,category_2,1076929,w4,NaN,4
3,category_1,1081321,w3,NaN,3
4,category_2,216151,w4,NaN,4
...,...,...,...,...,...
372,category_2,1101571,w1,NaN,1
373,category_2,1090258,w4,NaN,4
374,category_2,906595,w1,NaN,1
375,category_2,32245,w1,NaN,1


In [34]:
test_data.drop(['CategoryCode', 'Week'], axis = 1, inplace = True)

In [37]:
test_data.head()

,ItemCode,id
0,43738,4
1,1006090,1
2,1076929,4
3,1081321,3
4,216151,4


In [36]:
test_data.drop(['PredictedSales'], axis = 1, inplace = True)

In [43]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   ItemCode  377 non-null    int64
 1   id        377 non-null    int64
dtypes: int64(2)
memory usage: 6.0 KB


### Training full model

In [38]:
train_val_data.head()

,ItemCode,id,WeeklySales
0,174436,1,35
1,118033,1,7
2,20824,1,53
3,1061341,1,18
4,76399,1,21


In [44]:
xtr= melt2.drop(['WeeklySales'], axis=1)

In [45]:
ytr = melt2['WeeklySales'].values

In [46]:
mdl = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)

In [47]:
mdl.fit(xtr, ytr)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)

In [48]:
pred = mdl.predict(test_data)

In [51]:
pred

array([  8.658,   3.941,   5.834,  24.292,  17.273,   7.506,  10.571,
        11.538,   3.293,  12.429,   7.69 ,  16.367, 343.149,  52.769,
        24.801,  26.747,   8.695,   9.714,  12.718,   8.837,   4.605,
        10.068,  22.859,  18.335,  12.917,  47.739,  29.569,  20.353,
        12.1  ,  10.238,  15.013,  15.495,  14.218,  12.242,  24.212,
        89.039,  12.836,  30.342,   6.106, 725.297,  30.911,   4.103,
        30.895,   6.403,   6.005,   5.571,   3.789,   9.318,   9.601,
         8.345,  10.826,  17.564,  29.109,   6.533,   4.565,  88.238,
         5.97 ,  30.327,  96.599,  12.161,  16.609, 153.662,  21.011,
         4.854,  14.747,   6.11 ,   3.385,  31.053,  39.212,  35.614,
        33.59 ,   3.292,   5.174,   9.938,  48.621,  29.173,   6.778,
        46.957,  55.478,  65.954,  14.047,   3.032,  43.411,   6.123,
         5.951, 136.039,  55.569,  41.086,   6.844,  22.168,  12.34 ,
         6.012,  16.01 ,   7.865,  12.75 ,  11.037,  20.53 ,  14.343,
        12.155,  11.

In [52]:
pd.DataFrame(pred).to_csv('predictions_test.csv')